In [3]:
!pip install TTS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.1/604.1 kB 2.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 21.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 20.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 574.6/574.6 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.9 MB/s eta 0:00:00
     

In [4]:
dataset_path = '../input/the-lj-speech-dataset/LJSpeech-1.1/'

In [4]:
!pwd

/kaggle/working


In [5]:
%cd ..
%pwd

/kaggle


'/kaggle'

In [6]:
!ls

input  lib  working


In [7]:
%cd working

/kaggle/working


In [8]:
%pwd

'/kaggle/working'

In [5]:
output_path = './output/fakeljspeech/'

In [6]:
import os

# Trainer: Where the ✨️ happens.
# TrainingArgs: Defines the set of arguments of the Trainer.
from trainer import Trainer, TrainerArgs

# GlowTTSConfig: all model related values for training, validating and testing.
from TTS.tts.configs.glow_tts_config import GlowTTSConfig

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.glow_tts import GlowTTS
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

In [7]:
dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="metadata.csv", path=dataset_path
)

In [8]:
config = GlowTTSConfig(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=1000,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
)

In [ ]:
# INITIALIZE THE AUDIO PROCESSOR
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)

# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# If characters are not defined in the config, default characters are passed to the config
tokenizer, config = TTSTokenizer.init_from_config(config)

# LOAD DATA SAMPLES
# Each sample is a list of ```[text, audio_file_path, speaker_name]```
# You can define your custom sample loader returning the list of samples.
# Or define your custom formatter and pass it to the `load_tts_samples`.
# Check `TTS.tts.datasets.load_tts_samples` for more details.
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

# INITIALIZE THE MODEL
# Models take a config object and a speaker manager as input
# Config defines the details of the model like the number of layers, the size of the embedding, etc.
# Speaker manager is used by multi-speaker models.
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

# INITIALIZE THE TRAINER
# Trainer provides a generic API to train all the 🐸TTS models with all its perks like mixed-precision training,
# distributed training, etc.
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

# AND... 3,2,1... 🚀
trainer.fit()

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 | > Found 13100 files in /kaggle/input/the-lj-speech-dataset/LJSpeech-1.1


fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
 > Training Environment:
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 1
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 > Start Tensorboard: tensorboard --logdir=./output/fakeljspeech/run-February-11-2023_02+03PM-0000000

 > Model has 28610257 parameters

 > EPOCH: 0/1000
 --> ./output/fakeljspeech/run-February-11-2023_02+03PM-0000000


[*] Pre-computing phonemes...


  0%|          | 3/12969 [00:00<35:23,  6.10it/s]

ɪnstɛd əv weɪtɪŋ ðɛɹ, ɔzwɔld əpɛɹəntli wɛnt æz fɑɹ əweɪ æz hi kʊd ænd bɔɹdɪd ðə fɚst oʊk klɪf bʌs wɪt͡ʃ keɪm əlɔŋ
 [!] Character '͡' not found in the vocabulary. Discarding it.


...


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.00697 (-0.00352)
     | > avg_loss: -0.01722 (-0.00875)
     | > avg_log_mle: -0.24054 (-0.01106)
     | > avg_loss_dur: 0.22332 (+0.00231)

 > BEST MODEL : ./output/fakeljspeech/run-February-11-2023_02+03PM-0000000/best_model_25984.pth

 > EPOCH: 64/1000
 --> ./output/fakeljspeech/run-February-11-2023_02+03PM-0000000




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: gruut
	| > 3 not found characters:
	| > ͡
	| > “
	| > ”
| > Number of instances : 12969



 > TRAINING (2023-02-11 21:54:36) 


 | > Preprocessing samples
 | > Max text length: 188
 | > Min text length: 13
 | > Avg text length: 100.90014650319993
 | 
 | > Max audio length: 222643.0
 | > Min audio length: 24499.0
 | > Avg audio length: 144984.29755570978
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.



   --> STEP: 16/406 -- GLOBAL_STEP: 26000
     | > loss: -0.02454  (-0.04772)
     | > log_mle: -0.23433  (-0.23931)
     | > loss_dur: 0.20979  (0.19159)
     | > amp_scaler: 2048.00000  (2048.00000)
     | > grad_norm: 29.29327  (36.50051)
     | > current_lr: 0.00002 
     | > step_time: 0.64130  (0.59466)
     | > loader_time: 0.00950  (0.00772)


   --> STEP: 41/406 -- GLOBAL_STEP: 26025
     | > loss: -0.03381  (-0.03701)
     | > log_mle: -0.23240  (-0.23606)
     | > loss_dur: 0.19859  (0.19904)
     | > amp_scaler: 2048.00000  (2048.00000)
     | > grad_norm: 50.47242  (32.42015)
     | > current_lr: 0.00002 
     | > step_time: 0.60800  (0.60579)
     | > loader_time: 0.00390  (0.00797)


   --> STEP: 66/406 -- GLOBAL_STEP: 26050
     | > loss: -0.00996  (-0.03561)
     | > log_mle: -0.24516  (-0.23917)
     | > loss_dur: 0.23520  (0.20356)
     | > amp_scaler: 2048.00000  (2048.00000)
     | > grad_norm: 28.56041  (38.79124)
     | > current_lr: 0.00002 
     | > step_time: